In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df=pd.read_csv("app_data.log")

In [3]:
print(df.head(5))

         DATE      TIME APPLICATION
0  12-06-2023  12:05:01    nautilus
1  12-06-2023  12:05:03    nautilus
2  12-06-2023  12:05:04       gedit
3  12-06-2023  12:05:05       gedit
4  12-06-2023  12:05:06       gedit


# Data Cleaning

In [4]:
print(df.iloc[0,2])
print(df.loc[0,'APPLICATION'])

nautilus
nautilus


In [18]:
		"""Converting the data set into the fomat
		   DATE        START_TIME     END_TIME USAGE   APPLICATION
		   11-01-2022  11:49:42       11:50:33    52   thunar     """

cleaned_app_data="clean_app_data.csv"    #name of file in which w'll store clean data
new_df = pd.DataFrame(columns=["DATE", "START_TIME", "END_TIME","USAGE","APPLICATION"])

def time_difference(time1, time2):
    format_str = '%H:%M:%S'
    time_obj1 = datetime.strptime(time1, format_str)
    time_obj2 = datetime.strptime(time2, format_str)
    time_diff = time_obj2 - time_obj1
    return time_diff.total_seconds()
start_index=0
last_row_index = df.index[-1]
# print(last_row_index)
# try:
#     for index, row in df.iterrows():
#         if (df.iloc[index,2]==df.iloc[(index+1),2]):
#             time_diff= time_difference(df.iloc[index,1], df.iloc[(index+1),1])
#             if time_diff < 5 and (df.iloc[index,0]==df.iloc[(index+1),0]):
#                 pass
#         else:
#             new_df.loc[len(new_df)] = [df.iloc[index,0],df.iloc[start_index,1],df.iloc[index,1],time_diff,df.iloc[index,2]]
#             start_index=index
# except:
#     pass

# DATE      TIME APPLICATION
try:
    for index, row in df.iterrows():
        if (row["APPLICATION"]==df.iloc[(index+1),2]):
            time_diff= time_difference(row["TIME"], df.iloc[(index+1),1])
            if time_diff < 5 and (row["DATE"]==df.iloc[(index+1),0]):
                pass
        else:
            new_df.loc[len(new_df)] = [row["DATE"],df.iloc[start_index,1],row["TIME"],time_diff,row["APPLICATION"]]
            start_index=index
except:
    pass
print(new_df.head())
        

         DATE START_TIME  END_TIME  USAGE   APPLICATION
0  12-06-2023   12:05:01  12:05:03    2.0      nautilus
1  12-06-2023   12:05:03  12:05:07    1.0         gedit
2  12-06-2023   12:05:07  12:05:08    1.0      nautilus
3  12-06-2023   12:05:08  12:05:10    1.0           gjs
4  12-06-2023   12:05:10  12:05:17    1.0  sublime_text


In [19]:
def find_consecutive_rows(df, application, time_threshold):
    new_df = pd.DataFrame(columns=["DATE", "TIME", "DIFF", "APPLICATION"])
    start_index = 0
    for index, row in df.iterrows():
        if row["APPLICATION"] != application:
            new_df.loc[len(new_df)] = [row["DATE"], df.iloc[start_index, 1], row["TIME"], time_difference(row["TIME"], df.iloc[start_index, 1]), row["APPLICATION"]]
            start_index = index
        elif time_difference(row["TIME"], df.iloc[start_index, 1]) <= time_threshold and row["DATE"] == df.iloc[start_index, 0]:
            pass
        else:
            new_df.loc[len(new_df)] = [row["DATE"], df.iloc[start_index, 1], row["TIME"], time_difference(row["TIME"], df.iloc[start_index, 1]), row["APPLICATION"]]
            start_index = index

    return new_df

def time_difference(time1, time2):
    h1, m1, s1 = time1.split(":")
    h2, m2, s2 = time2.split(":")

    t1 = int(h1) * 3600 + int(m1) * 60 + int(s1)
    t2 = int(h2) * 3600 + int(m2) * 60 + int(s2)

    return t2 - t1

new_df = find_consecutive_rows(df, "APPLICATION", 5)

ValueError: cannot set a row with mismatched columns

# Show the pie chart of the current day computer usage in pie chart